In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

# Parámetros de conexión
user = 'jbpm'
password = 'jbpm'
host = 'localhost'
port = '5432'
database = 'jbpm'

# Crea la URL de conexión
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# Probar conexión: listar tablas
with engine.connect() as connection:
    result = connection.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'"))
    for row in result:
        # Imprimir usando el índice de la tupla
        print(row[0])  # row[0] es el valor de la primera columna (table_name)



In [ ]:
df = pd.read_sql("select name, processinstanceid, value from taskvariableimpl t ;", engine)
df.head()


In [ ]:
df_pivot = df.pivot(index='processinstanceid', columns='name', values='value').reset_index()
df_pivot.columns.name = None
df_pivot.head()


In [ ]:
df_pivot.to_sql(
    name='x_tabla_prueba',    # Nombre de la tabla en PostgreSQL
    con=engine,              # Conexión a la base de datos
    if_exists='append',      # Crea la tabla o la reemplaza si ya existe
    index=False              # No incluir el índice de pandas como columna
)

In [ ]:
df_verificacion = pd.read_sql('SELECT * FROM "public"."x_tabla_prueba"', engine)
df_verificacion.tail()

# UNION DE TABLAS POR ID, NOMBRE, INSTANCIA, TAREA, ETC

In [ ]:
# TABLA taskvariableimpl

df_vars = pd.read_sql("""
    SELECT processinstanceid, taskid, name as Campo_llenado, value as registro_de_campo
    FROM taskvariableimpl
""", engine)
df_vars.tail()


In [ ]:
# TABLA Tareas
df_tasks = pd.read_sql("""
    SELECT processinstanceid, id AS taskid, actualowner_id, formname as nombre_de_formulario
    FROM task
""", engine)
df_tasks.tail()

In [ ]:
# Paso 3: Unir variables con tareas (para saber quién hizo qué)
df_combined = pd.merge(df_tasks, df_vars, on=['processinstanceid', 'taskid'], how='left')
df_combined.tail()